# Predictions

In [ ]:
import re
from pathlib import Path
from functools import partial

import numpy as np
import pandas as pd

from pandas.io.formats.style import Styler

# Workaround for Firefox horizontal scroll for Dataframes
# https://github.com/jupyterlab/jupyterlab/issues/14625#issuecomment-1722137537
from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))

*Utils functions to generate highlighted tables*

In [ ]:
def highlight_predictions(dfs: Styler, axis: int = 0) -> Styler:
    """
    Highlight and format a DataFrame with style for prediction metrics.
    For all metrics predictions dataframe lower is better.

    Args:
        dfs (Styler): The pandas Styler object representing the DataFrame to be styled.
        axis (int, optional): The axis along which to apply the styling
        (0 for rows, 1 for columns). Defaults to 0.

    Returns:
        Styler: A Styler object with the specified styling applied.
    """
    dfs = dfs.background_gradient("Greens_r", axis=axis, low=1)
    dfs = dfs.highlight_min(props="font-weight: bold", axis=axis)
    return dfs


def table_html(df: pd.DataFrame, axis: int = 0) -> Styler:
    """Create HTML table with the predictions metrics for each experiment.
    This table is used to select the best experiment for each type.
    This will not be used in the final paper.
    """
    dfs = df.style.format(precision=3)
    dfs = highlight_predictions(dfs, axis=axis)
    return dfs


def table_tex(
    df: pd.DataFrame,
    tex_experiments_names: dict[str, str],
    tex_metrics_names: dict[str, str],
) -> str:
    """Create Tex table with the predictions metrics mean and std for
    each experiment type. This table will be include in the paper.
    """
    global std_index
    std_index = 0
    df_mean = df.groupby(df_experiments["name"]).mean()
    df_std = df.groupby(df_experiments["name"]).std().fillna(0)

    dfs = df_mean.T.style  # Traspose df scales better to more levels

    def fmt(value: float, precision: float):
        global std_index
        std = df_std.T.stack().values[std_index]
        std_index += 1
        return rf"{value:.{precision}f} \mdseries ± {std:.{precision}f}"

    dfs = highlight_predictions(dfs, axis=1)
    # It's ok to use precision=3 for all predictions metrics
    dfs = dfs.format(partial(fmt, precision=3))

    # Headers Style
    dfs = dfs.hide(names=True, axis=0)
    dfs = dfs.hide(names=True, axis=1)
    # dfs = dfs.format_index(tex_metrics_names, axis=0, level=1)
    # dfs = dfs.format_index(tex_experiments_names, axis=1)

    # Convert to TeX
    dfs = dfs.to_latex(
        hrules=True,
        column_format=f"X r *{{{len(df_mean)}}}{{c}}",
        convert_css=True,
        multirow_align="c",
        clines="skip-last;index",
    )
    dfs = dfs.replace(
        r"\cline{1-2}",  # clines don't work with colored background
        rf"\hhline{{{'-' * (len(df_mean) + 2)}}}",  # so replace with hhline
        len(hierarchy) - 2,  # on n occurences replace only the first n - 1
        # because the last one it replaced by ""
    ).replace(r"\cline{1-2}", "")
    # Exract tabular enviroment.
    # I prefer to work with tabular instead of table env for the following reasons:
    # - Better control of table position and dimensions
    # - Better control of caption position
    pattern = r"\\begin\{tabular\}(.*?)\\end\{tabular\}"
    dfs = re.search(pattern, dfs, re.DOTALL)
    assert dfs is not None, "Pattern not found"
    dfs = r"\begin{tabularx}{\linewidth}" + dfs.group(1) + r"\end{tabularx}"
    return dfs

In [ ]:
DATASET = "CIFAR100"

path_root = Path("..")
path_evals = path_root / "evals"
path_dataset = path_root / "datasets" / "datasets" / DATASET
path_results = path_evals / DATASET / "results"
path_results.mkdir(parents=True, exist_ok=True)
hierarchy = np.load(path_dataset / "hierarchy" / "hierarchy.npy")
metrics = ["error_rate", "hier_dist_mistake", "hier_dist"]

path_experiments = {
    "xe-onehot": sorted((path_evals / DATASET).glob("*_xe-onehot")),
    "xe-mbm": sorted((path_evals / DATASET).glob("*_xe-mbm-beta5.0")),
    "xe-b3p": sorted((path_evals / DATASET).glob("*_xe-b3p-beta0.4")),
    "cd-bd": sorted((path_evals / DATASET).glob("*_cd-barz-denzler")),
    "cd-desc": sorted((path_evals / DATASET).glob("*_cd-desc-pca-d100")),
}
tex_experiments_names = {
    "xe-onehot": r"XE One-hot",
    "xe-mbm": r"XE MBM $\beta\:5.0$",
    "xe-b3p": r"XE B3P $\beta\:0.4$",
    "cd-bd": r"CD BD",
    "cd-desc": r"CD Desc. $d\:100$",
}
tex_metrics_names = {
    "error_rate": "Error Rate",
    "hier_dist_mistake": "Hier. Dist. M.",
    "hier_dist": "Hier. Dist.",
}

df_experiments = pd.DataFrame(columns=["name"])
df_metrics = None
for name, paths in path_experiments.items():
    for path in paths:
        df_experiments.loc[path.name, "name"] = name
        path_predictions = path / "results" / "predictions.pkl"
        df_metrics = pd.concat([df_metrics, pd.read_pickle(path_predictions)])

df_mean = df_metrics.groupby(df_experiments["name"]).mean()
df_std = df_metrics.groupby(df_experiments["name"]).std().fillna(0)

with open(path_evals / DATASET / "results" / "predictions.tex", "w") as f:
    f.write(table_tex(df_metrics, tex_experiments_names, tex_metrics_names))

table_html(df_mean.T, axis=1)

In [ ]:
DATASET = "iNaturalist19"

path_root = Path("..")
path_evals = path_root / "evals"
path_dataset = path_root / "datasets" / "datasets" / DATASET
path_results = path_evals / DATASET / "results"
path_results.mkdir(parents=True, exist_ok=True)
hierarchy = np.load(path_dataset / "hierarchy" / "hierarchy.npy")
metrics = ["error_rate", "hier_dist_mistake", "hier_dist"]

path_experiments = {
    "xe-onehot": sorted((path_evals / DATASET).glob("*_xe-onehot")),
    "xe-mbm": sorted((path_evals / DATASET).glob("*_xe-mbm-beta15.0")),
    "xe-b3p": sorted((path_evals / DATASET).glob("*_xe-b3p-beta0.5")),
    "cd-bd": sorted((path_evals / DATASET).glob("*_cd-barz-denzler")),
    "cd-desc": sorted((path_evals / DATASET).glob("*_cd-desc-pca-d300")),
}
tex_experiments_names = {
    "xe-onehot": r"XE One-hot",
    "xe-mbm": r"XE MBM $\beta\:15.0$",
    "xe-b3p": r"XE B3P $\beta\:0.5$",
    "cd-bd": r"CD BD",
    "cd-desc": r"CD Desc. $d\:300$",
}
tex_metrics_names = {
    "error_rate": "Error Rate",
    "hier_dist_mistake": "Hier. Dist. M.",
    "hier_dist": "Hier. Dist.",
}

df_experiments = pd.DataFrame(columns=["name"])
df_metrics = None
for name, paths in path_experiments.items():
    for path in paths:
        df_experiments.loc[path.name, "name"] = name
        path_predictions = path / "results" / "predictions.pkl"
        df_metrics = pd.concat([df_metrics, pd.read_pickle(path_predictions)])


df_mean = df_metrics.groupby(df_experiments["name"]).mean()
df_std = df_metrics.groupby(df_experiments["name"]).std().fillna(0)

with open(path_evals / DATASET / "results" / "predictions.tex", "w") as f:
    f.write(table_tex(df_metrics, tex_experiments_names, tex_metrics_names))

table_html(df_mean.T, axis=1)